# Genie mutation filtering

I want to create a new table containing one row for each mutation in data_mutations_extended.txt and the following columns:
* Hugo_Symbol *- Official name of an allele.*
* SIFT_Prediction *- The prediced effects of amino acid substitutions on proteins using the SIFT (sorting intolerant from tolerant) algorithm*
* Polyphen_Prediction *- The prediced effects of amino acid substitutions on proteins using the PolyPhen (Polymorphism Phenotyping) algorithm*
* Variant_Classification *- Translational effect of variant allele*
* gnomAD_AMR_AF *- Allele frequency in gnomAD database whole-genome sequence data on Admixed American population.*
* Pathogenic *- Weather or not the mutation is pathogenic: 0 = is not, 1 = is.*

Unlike the other columns which can be found in the data_mutations_extended.txt file, the Pathogenic column should be calculated based on the values in the other columns.

SIFT_Prediction - https://doi.org/10.1093/nar/gks539

Polyphen_Prediction - https://doi.org/10.1002/0471142905.hg0720s76


## Setup

In [1]:
library(ggplot2)

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”


## Get data

In [2]:
mutation_data <- read.table("../../data/genie_v15/data_mutations_extended.txt", sep="\t", quote="", head=TRUE)

dim(mutation_data)
head(mutation_data)

[1] 1840311      64

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,⋯,FILTER,Polyphen_Prediction,Polyphen_Score,SIFT_Prediction,SIFT_Score,SWISSPROT,n_depth,t_depth,Annotation_Status,mutationInCis_Flag
,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<chr>,<dbl>,<lgl>,<int>,<int>,<chr>,<chr>
1,KRAS,3845,JHU,GRCh37,12,25398285,25398285,+,missense_variant,Missense_Mutation,⋯,PASS,probably_damaging,0.991,deleterious,0.04,NA,NA,1623,SUCCESS,False
2,BRAF,673,JHU,GRCh37,7,140453136,140453136,+,missense_variant,Missense_Mutation,⋯,PASS,probably_damaging,0.963,deleterious,0.00,NA,NA,1031,SUCCESS,False
3,EGFR,1956,JHU,GRCh37,7,55249071,55249071,+,missense_variant,Missense_Mutation,⋯,PASS,probably_damaging,1.000,deleterious,0.00,NA,NA,692,SUCCESS,False
4,TP53,7157,JHU,GRCh37,17,7577120,7577120,+,missense_variant,Missense_Mutation,⋯,PASS,possibly_damaging,0.643,tolerated,0.13,NA,NA,930,SUCCESS,False
5,NRAS,4893,JHU,GRCh37,1,115256529,115256529,+,missense_variant,Missense_Mutation,⋯,PASS,benign,0.251,tolerated,0.06,NA,NA,2277,SUCCESS,False
6,PIK3CA,5290,JHU,GRCh37,3,178952085,178952085,+,missense_variant,Missense_Mutation,⋯,PASS,benign,0.085,tolerated,0.11,NA,NA,1064,SUCCESS,False


Now that we have the data, we want to do three things.

1) Choose a subset of the columns

2) Add a Population column

There are five centers not located in America:
* NKI - Netherlands
* GRCC - France
* UHN - Canada
* CRUK - England
* VHIO - Spain

Of these five, all except Canada can be categorised as non-Finish Europeans.

3) Add a Pathogenic column initiated with only 0s.

At the end of the scrit we want the ones that are pathogenic to have a value of 1, and to evaluate that based on the other columns.

In [3]:
# Choosing the columns
mutation_pathogen_filter <- mutation_data[c('Hugo_Symbol', 'Tumor_Sample_Barcode', 'Center', 'SIFT_Prediction', 'Polyphen_Prediction', 'Variant_Classification', 'gnomAD_AMR_AF', 'gnomAD_NFE_AF')]

# Adding the new column and giving population values
mutation_pathogen_filter['Population'] <- 'AMR'
mutation_pathogen_filter['Population'][mutation_pathogen_filter['Center'] == 'NKI'] <- 'NFE'
mutation_pathogen_filter['Population'][mutation_pathogen_filter['Center'] == 'GRCC'] <- 'NFE'
mutation_pathogen_filter['Population'][mutation_pathogen_filter['Center'] == 'CRUK'] <- 'NFE'
mutation_pathogen_filter['Population'][mutation_pathogen_filter['Center'] == 'VHIO'] <- 'NFE'

# Removing the Center column
mutation_pathogen_filter <- subset(mutation_pathogen_filter, select = -Center )

# Examining the new dataframe
dim(mutation_pathogen_filter)
head(mutation_pathogen_filter)

[1] 1840311       8

,Hugo_Symbol,Tumor_Sample_Barcode,SIFT_Prediction,Polyphen_Prediction,Variant_Classification,gnomAD_AMR_AF,gnomAD_NFE_AF,Population
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,KRAS,GENIE-JHU-00006-00185,deleterious,probably_damaging,Missense_Mutation,0.00000e+00,0.00000e+00,AMR
2,BRAF,GENIE-JHU-00006-00185,deleterious,probably_damaging,Missense_Mutation,0.00000e+00,0.00000e+00,AMR
3,EGFR,GENIE-JHU-00006-00185,deleterious,probably_damaging,Missense_Mutation,2.89101e-05,3.51673e-05,AMR
4,TP53,GENIE-JHU-00006-00185,tolerated,possibly_damaging,Missense_Mutation,0.00000e+00,2.64271e-05,AMR
5,NRAS,GENIE-JHU-00006-00185,tolerated,benign,Missense_Mutation,NA,NA,AMR
6,PIK3CA,GENIE-JHU-00006-00185,tolerated,benign,Missense_Mutation,NA,NA,AMR


## Examine Data

#### Population

In [4]:
# How many mutations belong to each population?
table(mutation_pathogen_filter$Population)


    AMR     NFE 
1812075   28236 

#### SIFT Prediction

In [5]:
# Setting missing predictions to be NA
mutation_pathogen_filter['SIFT_Prediction'][mutation_pathogen_filter['SIFT_Prediction'] == ''] <- NA

# How many mutations are in each category?
table(mutation_pathogen_filter$SIFT_Prediction)


               deleterious deleterious_low_confidence 
                    649421                      65371 
                 tolerated   tolerated_low_confidence 
                    424199                      58876 

In [6]:
# How much of the data has a SIFT prediction?
((649421 + 65371 + 424199 + 58876)/1840311)*100

[1] 65.09047

65% of the data has a SIFT_Prediction score.

#### Polyphen prediction

In [7]:
# Setting missing and unknown predictions to be NA
mutation_pathogen_filter['Polyphen_Prediction'][mutation_pathogen_filter['Polyphen_Prediction'] == ''] <- NA
mutation_pathogen_filter['Polyphen_Prediction'][mutation_pathogen_filter['Polyphen_Prediction'] == 'unknown'] <- NA

# How many mutations are in each category?
table(mutation_pathogen_filter$Polyphen_Prediction)


           benign possibly_damaging probably_damaging 
           525778            200715            476703 

In [8]:
# How much of the data has a Polyphen prediction?
((525778 + 200715 + 476703)/1840311)*100

[1] 65.38004

65% of the data has a Polyphen_Prediction

#### Missing predictions

In [8]:
# How many mutations have neither prediction?
missing_pred <- length(which(is.na(mutation_pathogen_filter$Polyphen_Prediction) & is.na(mutation_pathogen_filter$SIFT_Prediction)))

(missing_pred/1840311)*100

[1] 34.60002

34% of the data has no predicted outcome.

#### Variant Classification

In [9]:
table(mutation_pathogen_filter$Variant_Classification)


               3'Flank                  3'UTR                5'Flank 
                  6991                   3873                  25546 
                 5'UTR        Frame_Shift_Del        Frame_Shift_Ins 
                  3801                 102725                  46969 
          In_Frame_Del           In_Frame_Ins                 Intron 
                 24542                   9225                  54127 
     Missense_Mutation      Nonsense_Mutation       Nonstop_Mutation 
               1209408                 127372                    924 
                   RNA                 Silent          Splice_Region 
                  2090                 124278                  51972 
           Splice_Site Translation_Start_Site 
                 44459                   2009 

In [10]:
# How many mutations lack a variant classification
missing_variant <- length(which(is.na(mutation_pathogen_filter$Variant_Classification)))
(missing_variant/1840311)*100

[1] 0

All mutations have a Variant Classification. 

#### HUGO symbl

In [11]:
# How many unique Hugo symbols are there?
length(unique(mutation_pathogen_filter$Hugo_Symbol))

[1] 1682

The 1840311 data points are divided into 1682 types of mutations.

#### gnomAD_AF

In [12]:
# How many common allele frequencies are there in the AMR populations?

# 1. How many common are there
common_gnomAD_AMR <- length(which(mutation_pathogen_filter$gnomAD_AMR_AF>0.01))
# 2. How many known gnomAD values are there?
missing_gnomAD_AMR <- length(which(is.na(mutation_pathogen_filter$gnomAD_AMR_AF)))
not_missing_gnomAD_AMR <- (1840311 - missing_gnomAD_AMR)
# 3. What percentage of known allele frequencies are common across the populations?
(common_gnomAD_AMR/not_missing_gnomAD_AMR)*100


# How many common allele frequencies are there in the NFE populations?

# 1. How many common are there
common_gnomAD_NFE <- length(which(mutation_pathogen_filter$gnomAD_NFE_AF>0.01))
# 2. How many known gnomAD values are there?
missing_gnomAD_NFE <- length(which(is.na(mutation_pathogen_filter$gnomAD_NFE_AF)))
not_missing_gnomAD_NFE <- (1840311 - missing_gnomAD_NFE)
# 3. What percentage of known allele frequencies are common across the populations?
(common_gnomAD_NFE/not_missing_gnomAD_NFE)*100



# How much missing data is missing all gnomAD values?
missing_gnomAD <- length(which(is.na(mutation_pathogen_filter$gnomAD_AMR_AF) & is.na(mutation_pathogen_filter$gnomAD_NFE_AF)))

(missing_gnomAD/1840311)*100

[1] 0.03778699

[1] 0.02083232

[1] 83.30635

In the AMR and NFE population, there are 0.04% and 0.02% that have common mutations, resectively.

83% of the data does not have any gnomAD value. This is because the mutations have not been seen in the population.

## The filter

If the mutation is common in the population >0.01, it is not a pathogen.

If he mutation significantly impacts a gene, like a Frame-shift mutation, Nonsense-mutation, Splicing-mutation or Tanslation mutation, it is a pathogen.

If the mutation is a Missense mutation, the Polyphen and SIFT prediction must both be confident in it being damaging/deleterious to be a pathogen.

This is the background for out basic idea:

    bad_mutations = c(Frame_Shift_Del, 
                        Frame_Shift_Ins,
                        Nonsense_Mutation, #- creates a nonsense or stop codon
                        Splice_Region, #- Mutations near the splice site
                        Splice_Site,
                        Translation_Start_Site,
                        Nonstop_Mutation #- does not stop
                        )

    if gnomAD_pop_AF < 0.01:
        if Variant_Class in bad_mutations :
            Pathogen <- 1
        if Variant_Class == 'Missense':
            if Polyphen_Prediction == 'probably_damaging' & SIFT_Prediction == 'deleterious'
                Pathogen <- 1
                 


In [14]:
# We create the Pathogen column with a default of not-pathogenic
mutation_pathogen_filter$Pathogen <- 0

# We define a list of mutations we know to be pathogenic
bad_mutations <- c('Frame_Shift_Del', 'Frame_Shift_Ins', 'Nonsense_Mutation', 'Splice_Site', 'Translation_Start_Site', 'Nonstop_Mutation')

# Find all rows where the population gnomAD allele frequency is bellow 0.01
filter1 <- which(mutation_pathogen_filter$Population == 'AMR' & mutation_pathogen_filter$gnomAD_AMR_AF <= 0.01)
filter2 <- which(mutation_pathogen_filter$Population == 'NFE' & mutation_pathogen_filter$gnomAD_NFE_AF <= 0.01)
# And all rows where there is no gnomAD value
filter3 <- which(is.na(mutation_pathogen_filter$gnomAD_NFE_AF) & is.na(mutation_pathogen_filter$gnomAD_AMR_AF))

# Combine the lists
filter <- c(filter1, filter2, filter3)

# We go through all rows with uncommon mutations
for (i in filter) {
    # If the mutation is a bad mutaion...
    if (mutation_pathogen_filter$Variant_Classification[i] %in% bad_mutations) {
        # ... It is pathogenic
        mutation_pathogen_filter$Pathogen[i] <- 1
    }
    # If the mutation is a missense mutation...
    if (mutation_pathogen_filter$Variant_Classification[i] == 'Missense_Mutation') {
        # ... And both Polyphen and SIFT prediction is probably damaging/deleterious...
        if (!is.na(mutation_pathogen_filter$Polyphen_Prediction[i]) & !is.na(mutation_pathogen_filter$SIFT_Prediction[i]) & mutation_pathogen_filter$Polyphen_Prediction[i] == 'probably_damaging' & mutation_pathogen_filter$SIFT_Prediction[i] == 'deleterious') {
            # ... It is pathogenic
            mutation_pathogen_filter$Pathogen[i] <- 1
        }
    # If not, it is not pathogenic (default value = 0)
    }
}

In [15]:
# How many have been evaluated as not-/pathogenic?
table(mutation_pathogen_filter$Pathogen)


      0       1 
1105511  734800 

In [18]:
(734800/1840311)*100

[1] 39.92803

## Results

There are 734.800 pathogens (~40% of the total dataset). The filter table is saved in a file (takes ~15 minutes to run the last part of the code).

In [17]:
write.csv(mutation_pathogen_filter, "../../derived_data/genie_v15/mutation_pathogen_filter.csv", row.names=TRUE)